In [ ]:
from SBM_attributed import *
import torch
import networkx as nx
from divergences import * 
from copy import deepcopy
from torchmin import minimize_constr
from sklearn.metrics import rand_score, calinski_harabasz_score, mutual_info_score, accuracy_score

In [ ]:
# !pip install pytorch-minimize

In [ ]:
### Generate benchmark
P = np.array([[0.8, 0.2, 0.3],[0.2, 0.7, 0.4],[0.3, 0.4, 0.6]])
c = 3
n = 100
N = c*n
delta = 10
d = 2
dim = c*d
_,A,X = generate_benchmark(P,c,delta=delta,n=n,dim=d,sample_along_direction=True)
A = torch.tensor(A,dtype=torch.float32)
X = torch.tensor(X,dtype=torch.float32)
norm = -1

In [ ]:
### initialize variables
W = torch.zeros((N,c),dtype=torch.float)
indexes = torch.randint(low=0,high=c,size=(N,))
with torch.no_grad():
    for index,col in enumerate(indexes):
        W[index].index_fill_(0, col, 1)
WW_inv = torch.inverse(W.T@W)
B = WW_inv@W.T@A@W@WW_inv
mu = WW_inv@W.T@X
x_0 = torch.zeros(N*c + c*c + c*dim,requires_grad=False)
with torch.no_grad():
    x_0[:] = torch.hstack([W.flatten(),B.flatten(),mu.flatten()])

In [ ]:
x_0.requires_grad

In [ ]:
phi_net = get_phi("euclidean",elementwise=True)[0]
phi_data = get_phi("euclidean")

In [ ]:
def obj_func(x):
    W = x[:N*c].view(N, c)
    B = x[N*c:N*c+c*c].view(c, c)
    mu = x[N*c+c*c:].view(c,dim)
    net_divergence = torch.sum(phi_net(A,W@B@W.T), axis=1)
    data_divergence = torch.sum(torch.multiply(W, pairwise_bregman(X, mu, phi_data)),axis=1)
    K = torch.stack((net_divergence,data_divergence),axis=-1)
    return torch.sum(torch.pow(torch.sum(torch.pow(K, norm), axis=1),1/norm))
    #     return torch.linalg.matrix_norm(K,ord=-np.inf,keepdim=False)

In [ ]:
#sum of W matrix row wise is one 
def make_constraints(N,c,dim):
    C = torch.cat([torch.kron(torch.eye(N),torch.ones(c)),torch.zeros((N,c*c + c*dim))],axis=1)
    constr = dict(
        fun = lambda x: torch.norm(C@x - torch.ones(N),1), 
        lb=0, ub=0
    )
    return constr

def make_bounds(N,c,dim):
    W_B_bounds = [(0,1)]*(N*c + c*c)
    mu_bounds = [(-np.inf,np.inf)]*(c*dim)
    bounds = np.vstack([W_B_bounds,mu_bounds])
    bounds_dict = dict(lb=bounds[:,0],ub=bounds[:,1])
    return bounds_dict
bounds = make_bounds(N,c,dim)
constraints = make_constraints(N,c,dim)

In [ ]:
obj_func(x_0)

In [ ]:
obj_func(x_0).requires_grad

In [ ]:
res = minimize_constr(
    obj_func, x_0, 
    max_iter=15000,
    constr=constraints,
    bounds=bounds
)

In [ ]:
bounds['ub'][N*c:N*c+c*c]

In [ ]:
W = res.x[:N*c].reshape((N, c)).detach().numpy()
B = res.x[N*c:N*c+c*c].reshape((c, c)).detach().numpy()
mu = res.x[N*c+c*c:].reshape((c, dim)).detach().numpy()

In [ ]:
print(B,mu)

In [ ]:
WW_inv = np.linalg.inv(W.T@W)
B = WW_inv@W.T@A.detach().numpy()@W@WW_inv
mu = WW_inv@W.T@X.detach().numpy()

In [ ]:
pred_labels = np.argmax(W, axis=1)
true_labels = [0]*n + [1]*n + [2]*n
print(rand_score(true_labels, pred_labels),mutual_info_score(true_labels, pred_labels))

In [ ]:
print(B,mu,P)